In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [2]:
df = np.load("datasets/df.npz")

X = df["x"]
Y = df["y"]
Z = df["z"]

df = None
print(X.shape)

(1962, 224, 224)


In [19]:
def make_model():
    model = models.Sequential()
    shape=(X.shape[1], X.shape[2], 1)
    for i in range(0, 6):
        if i == 0:
            model.add(layers.Conv2D(32 + 2 ** (4 + i), (3, 3), strides=(2,2), activation='selu', input_shape=shape))
        else:
            model.add(layers.Conv2D(32 + 2 ** (4 + i), (3, 3), strides=(2,2), activation='relu'))
            

    model.add(layers.Flatten())
    model.add(layers.Dense(2, activation="relu"))

    model.summary()

    model.compile(optimizer='adam',
                  loss="mse",
                  metrics="mse")
    return model

model = make_model()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_37 (Conv2D)          (None, 111, 111, 48)      480       
                                                                 
 conv2d_38 (Conv2D)          (None, 55, 55, 64)        27712     
                                                                 
 conv2d_39 (Conv2D)          (None, 27, 27, 96)        55392     
                                                                 
 conv2d_40 (Conv2D)          (None, 13, 13, 160)       138400    
                                                                 
 conv2d_41 (Conv2D)          (None, 6, 6, 288)         415008    
                                                                 
 conv2d_42 (Conv2D)          (None, 2, 2, 544)         1410592   
                                                                 
 flatten_7 (Flatten)         (None, 2176)             

In [20]:
history = model.fit(X, Y, epochs = 20, batch_size = 16, validation_split = 0.7)

Epoch 1/20
37/37 [==============================] - 11s 286ms/step - loss: 0.1418 - mse: 0.1418 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 2/20
37/37 [==============================] - 10s 283ms/step - loss: 0.0359 - mse: 0.0359 - val_loss: 0.0171 - val_mse: 0.0171
Epoch 3/20
37/37 [==============================] - 11s 291ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0155 - val_mse: 0.0155
Epoch 4/20
37/37 [==============================] - 10s 287ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.0142 - val_mse: 0.0142
Epoch 5/20
37/37 [==============================] - 11s 300ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0131 - val_mse: 0.0131
Epoch 6/20
37/37 [==============================] - 11s 299ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 7/20
37/37 [==============================] - 11s 288ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 8/20
37/37 [==============================] - 11s 293ms/step - 

In [21]:
model.save_weights("weights/eye.h5")

In [22]:
preds = model.predict(X, verbose=0)

In [23]:
for i, pred in enumerate(preds):
    print(pred, Y[i])


[0.51234066 0.35806587] [0.47526042 0.45601852]
[0.5397563 0.4030814] [0.47526042 0.45601852]
[0.4897374  0.41037738] [0.47526042 0.45601852]
[0.4897374  0.41037738] [0.47526042 0.45601852]
[0.5128262  0.40885884] [0.47526042 0.45601852]
[0.5062202  0.43339303] [0.47526042 0.45601852]
[0.5062202  0.43339303] [0.47526042 0.45601852]
[0.5046636 0.4248957] [0.47526042 0.45601852]
[0.5051488  0.42061204] [0.47526042 0.45601852]
[0.5051488  0.42061204] [0.47526042 0.45601852]
[0.51694864 0.43717632] [0.47526042 0.45601852]
[0.5371899 0.4248748] [0.47526042 0.45601852]
[0.5371899 0.4248748] [0.47526042 0.45601852]
[0.5094012 0.4457435] [0.47526042 0.45601852]
[0.5257198  0.45784888] [0.47526042 0.45601852]
[0.5257198  0.45784888] [0.47526042 0.45601852]
[0.49748832 0.4468581 ] [0.47526042 0.45601852]
[0.534981   0.42881787] [0.47526042 0.45601852]
[0.534981   0.42881787] [0.47526042 0.45601852]
[0.5384311  0.41013333] [0.47526042 0.45601852]
[0.5384311  0.41013333] [0.47526042 0.45601852]
[0

[0.26507398 0.0463102 ] [0.37369792 0.01273148]
[0.26507398 0.0463102 ] [0.36328125 0.01157407]
[0.24919112 0.05171113] [0.34505208 0.00810185]
[0.25841236 0.03664979] [0.33528646 0.00694444]
[0.25841236 0.03664979] [0.33138021 0.00694444]
[0.25755787 0.07546659] [0.328125   0.00694444]
[0.25755787 0.07546659] [0.3203125  0.00694444]
[0.21327025 0.10344931] [0.31510417 0.00810185]
[0.22529732 0.07003804] [0.30924479 0.00925926]
[0.22529732 0.07003804] [0.3046875  0.01041667]
[0.22664233 0.03944089] [0.29882812 0.01157407]
[0.22664233 0.03944089] [0.29427083 0.01157407]
[0.22180675 0.07370432] [0.29427083 0.01157407]
[0.2192496  0.06994817] [0.29427083 0.01157407]
[0.2192496  0.06994817] [0.29427083 0.01157407]
[0.2065179  0.08800066] [0.29427083 0.01157407]
[0.2104039  0.08171552] [0.29427083 0.01157407]
[0.2104039  0.08171552] [0.27408854 0.00925926]
[0.18397178 0.09331619] [0.25130208 0.00810185]
[0.18397178 0.09331619] [0.24088542 0.00810185]
[0.18916552 0.10232037] [0.23567708 0.00